In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 717.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.4/869.4 kB 6.7 MB/s eta 0:00:00


In [4]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov9c.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

100%|██████████| 49.4M/49.4M [00:00<00:00, 214MB/s]


In [5]:
tracker=Tracker()
count=0

In [6]:
import cv2
from google.colab.patches import cv2_imshow

In [7]:
import cv2
import numpy as np

def cizgi(frame, baslangic, bitis, color=(0, 0, 255), alpha=0.4):
    # Dikdörtgenin koordinatlarını belirle
    start_point = (baslangic[0], baslangic[1] - 15)  # Üst sol köşe
    end_point = (bitis[0], bitis[1] + 15)            # Alt sağ köşe

    # Dikdörtgeni çizmek için bir katman oluştur
    overlay = frame.copy()

    # Dikdörtgeni overlay üzerinde çiz
    cv2.rectangle(overlay, start_point, end_point, color, -1)

    # Şeffaf dikdörtgeni ana görüntüyle birleştir
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

In [8]:
def draw_rectangle_with_text(frame, start_point, end_point, text, font=cv2.FONT_HERSHEY_PLAIN, font_scale=1.5, font_thickness=2, text_color=(255, 255, 255), rect_color=(112, 138, 144)):
    # Draw the rectangle
    cv2.rectangle(frame, start_point, end_point, rect_color, -1)

    # Calculate text size and position
    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_x = start_point[0] + (end_point[0] - start_point[0] - text_width) // 2
    text_y = start_point[1] + (end_point[1] - start_point[1] + text_height) // 2
    text_y += text_height // 2

    # Put the text on the frame
    cv2.putText(frame, text, (text_x, text_y), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

In [1]:
import cv2
import pandas as pd
import time

# Initialize VideoCapture
input_path = '/content/denemevideo - Trim.mp4'
cap = cv2.VideoCapture(input_path)

# Dictionaries to track object positions, speeds, and crossing events
down = {}
up = {}
speed_dict = {}
counter_down = set()
counter_up = set()
time_down = {}
time_up = {}
speed_data = {}

count = 0
fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frames per second of the video

# Define the real-world scale (in meters per pixel)
scale_meters_per_pixel = 0.05  # Example value, adjust based on your video

# Number of frames to process
max_frames = 15  # Only process 3 frames

down_coordinat=900
up_coordinat=778

while True:
    ret, frame = cap.read()
    if not ret or count >= max_frames:
        break
    count += 1

    results = model.predict(frame)

    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    detection_list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            detection_list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(detection_list)

    # Draw bounding boxes and IDs
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int((x3 + x4) / 2)
        cy = int((y3 + y4) / 2)

        # Draw bounding box and ID
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
        cv2.circle(frame, (cx, cy), 2, (0, 0, 255), -1)
        cv2.putText(frame, str(id), (x3 + 5, y3 + 15), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 255, 255), 2)

        # Calculate speed in pixels/s
        if id in speed_dict:
            prev_cx, prev_cy, prev_count = speed_dict[id]
            distance = ((cx - prev_cx) ** 2 + (cy - prev_cy) ** 2) ** 0.5
            time_elapsed = (count - prev_count) / fps
            speed_px_per_s = distance / time_elapsed if time_elapsed > 0 else 0

            # Convert speed to km/h
            speed_kmh = speed_px_per_s * scale_meters_per_pixel * 3.6
            speed_dict[id] = (cx, cy, count)
        else:
            speed_kmh = 0
            speed_dict[id] = (cx, cy, count)

        # Down line detection

        y_down = down_coordinat
        offset_down = 15
        if y_down < (cy + offset_down) and y_down > (cy - offset_down):
            if id in down:
                if down[id] < cy:
                    counter_down.add(id)

                    cizgi(frame, (378, down_coordinat), (978, down_coordinat), color=(255, 255, 255), alpha=0.4)

                    elapsed_time = time.time() - time_down[id]
                    distance = 10  # Distance between the lines in meters
                    speed_ms = distance / elapsed_time
                    speed_kmh = speed_ms * 3.6
                    if speed_kmh > 0:  # Display speed only if it's greater than 0
                        speed_data[id] = speed_kmh
                        cv2.putText(frame, f'{speed_kmh:.2f} km/h', (x3, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                        cv2.rectangle(frame, (x3, y3), (x4, y4), (255, 255, 0), 2)

            down[id] = cy
            time_down[id] = time.time()  # Record the time when the object crosses the line

        # Up line detection

        y_up = up_coordinat
        offset_up = 15
        if y_up < (cy + offset_up) and y_up > (cy - offset_up):
            if id in up:
                if up[id] > cy:
                    counter_up.add(id)

                    cizgi(frame, (1054, up_coordinat), (1494, up_coordinat), color=(255, 255, 255), alpha=0.4)

                    elapsed_time = time.time() - time_up[id]
                    distance = 10  # Distance between the lines in meters
                    speed_ms = distance / elapsed_time
                    speed_kmh = speed_ms * 3.6
                    if speed_kmh > 0:  # Display speed only if it's greater than 0
                        speed_data[id] = speed_kmh
                        cv2.putText(frame, f'{speed_kmh:.2f} km/h', (x3, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                        cv2.rectangle(frame, (x3, y3), (x4, y4), (255, 255, 0), 2)

            up[id] = cy
            time_up[id] = time.time()  # Record the time when the object crosses the line



    # Drawing the lines and text
    cizgi(frame, (378, down_coordinat), (978, down_coordinat), color=(0, 0, 255), alpha=0.4)
    cizgi(frame, (1054, up_coordinat), (1494, up_coordinat), color=(255, 0, 255), alpha=0.4)

    downwards = len(counter_down)
    upwards = len(counter_up)


    # Dikdörtgenin köşe koordinatları
    top_left = (0, 0)
    bottom_right = (400, 160)
    start_point = (1520, 0)   # Sol üst köşe
    end_point = (1920, 160)    # Sağ alt köşe
    cv2.rectangle(frame, start_point, end_point, (112, 138, 144),-1)
    cv2.rectangle(frame, top_left, bottom_right, (250, 128, 114),-1)


    draw_rectangle_with_text(frame, top_left, bottom_right, f'Going Down : {downwards}')
    draw_rectangle_with_text(frame, start_point, end_point, f'Going Up : {upwards}')


    print(speed_data)

    # Show each frame
    cv2_imshow(frame)  # Use cv2_imshow instead of cv2.imshow

    # Break loop with 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print("Frames displayed.")


Frames displayed.


VİDEO

In [14]:
import cv2
import pandas as pd
import time

# Initialize VideoCapture
input_path = '/content/denemevideo.mp4'
cap = cv2.VideoCapture(input_path)

# Initialize VideoWriter
output_path = '/content/mobesekayit.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frames per second of the video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Dictionaries to track object positions, speeds, and crossing events
down = {}
up = {}
speed_dict = {}
counter_down = set()
counter_up = set()
time_down = {}
time_up = {}
speed_data = {}

count = 0

# Define the real-world scale (in meters per pixel)
scale_meters_per_pixel = 0.05  # Example value, adjust based on your video


down_coordinat = 900
up_coordinat = 778

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1

    results = model.predict(frame)

    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    detection_list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            detection_list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(detection_list)

    # Draw bounding boxes and IDs
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int((x3 + x4) / 2)
        cy = int((y3 + y4) / 2)

        # Draw bounding box and ID
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
        cv2.circle(frame, (cx, cy), 2, (0, 0, 255), -1)
        cv2.putText(frame, str(id), (x3 + 5, y3 + 15), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 255, 255), 2)

        # Calculate speed in pixels/s
        if id in speed_dict:
            prev_cx, prev_cy, prev_count = speed_dict[id]
            distance = ((cx - prev_cx) ** 2 + (cy - prev_cy) ** 2) ** 0.5
            time_elapsed = (count - prev_count) / fps
            speed_px_per_s = distance / time_elapsed if time_elapsed > 0 else 0

            # Convert speed to km/h
            speed_kmh = speed_px_per_s * scale_meters_per_pixel * 3.6
            speed_dict[id] = (cx, cy, count)
        else:
            speed_kmh = 0
            speed_dict[id] = (cx, cy, count)

        # Down line detection
        y_down = down_coordinat
        offset_down = 15
        if y_down < (cy + offset_down) and y_down > (cy - offset_down):
            if id in down:
                if down[id] < cy:
                    counter_down.add(id)

                    cizgi(frame, (378, down_coordinat), (978, down_coordinat), color=(255, 255, 255), alpha=0.4)

                    elapsed_time = time.time() - time_down[id]
                    distance = 10  # Distance between the lines in meters
                    speed_ms = distance / elapsed_time
                    speed_kmh = speed_ms * 3.6
                    if speed_kmh > 0:  # Display speed only if it's greater than 0
                        speed_data[id] = speed_kmh
                        cv2.putText(frame, f'{speed_kmh:.2f} km/h', (x3, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                        cv2.rectangle(frame, (x3, y3), (x4, y4), (255, 255, 0), 2)

            down[id] = cy
            time_down[id] = time.time()  # Record the time when the object crosses the line

        # Up line detection
        y_up = up_coordinat
        offset_up = 15
        if y_up < (cy + offset_up) and y_up > (cy - offset_up):
            if id in up:
                if up[id] > cy:
                    counter_up.add(id)

                    cizgi(frame, (1054, up_coordinat), (1494, up_coordinat), color=(255, 255, 255), alpha=0.4)

                    elapsed_time = time.time() - time_up[id]
                    distance = 10  # Distance between the lines in meters
                    speed_ms = distance / elapsed_time
                    speed_kmh = speed_ms * 3.6
                    if speed_kmh > 0:  # Display speed only if it's greater than 0
                        speed_data[id] = speed_kmh
                        cv2.putText(frame, f'{speed_kmh:.2f} km/h', (x3, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                        cv2.rectangle(frame, (x3, y3), (x4, y4), (255, 255, 0), 2)

            up[id] = cy
            time_up[id] = time.time()  # Record the time when the object crosses the line

    # Drawing the lines and text
    cizgi(frame, (378, down_coordinat), (978, down_coordinat), color=(0, 0, 255), alpha=0.4)
    cizgi(frame, (1054, up_coordinat), (1494, up_coordinat), color=(255, 0, 255), alpha=0.4)

    downwards = len(counter_down)
    upwards = len(counter_up)

    # Dikdörtgenin köşe koordinatları
    top_left = (0, 0)
    bottom_right = (400, 160)
    start_point = (1520, 0)   # Sol üst köşe
    end_point = (1920, 160)    # Sağ alt köşe
    cv2.rectangle(frame, start_point, end_point, (112, 138, 144), -1)
    cv2.rectangle(frame, top_left, bottom_right, (250, 128, 114), -1)

    draw_rectangle_with_text(frame, top_left, bottom_right, f'Going Down : {downwards}')
    draw_rectangle_with_text(frame, start_point, end_point, f'Going Up : {upwards}')

    print(speed_data)

    # Write frame to video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print("Video saved.")



0: 384x640 14 cars, 2 trucks, 188.6ms
Speed: 2.5ms preprocess, 188.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 cars, 3 trucks, 208.8ms
Speed: 2.2ms preprocess, 208.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 cars, 2 trucks, 1 traffic light, 206.0ms
Speed: 2.0ms preprocess, 206.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 cars, 3 trucks, 204.1ms
Speed: 2.1ms preprocess, 204.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 2 trucks, 218.9ms
Speed: 2.0ms preprocess, 218.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 cars, 2 trucks, 212.9ms
Speed: 2.2ms preprocess, 212.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 2 trucks, 204.6ms
Speed: 2.2ms preprocess, 204.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 cars, 2 t

In [15]:
counter_down = set()
counter_up = set()